In [42]:
%run 0.0_Dependance.ipynb

In [43]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

In [44]:
param_grid_initial = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [50, 100, 250, 500, 1000],
    
    # Profondeur de chaque arbre
    'max_depth': [3, 6, 9, 12],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0, 0.1, 0.5, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.5, 0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.5, 0.75, 1],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [1, 5, 10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0, 0.1, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1, 10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}

In [45]:
param_grid_ameliore = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [400,500,600,700,800,900],
    
    # Profondeur de chaque arbre
    'max_depth': [4,5,6],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.05,0.06,0.07,0.08,0.09,0.1],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0.5,0.7, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.2,0.4,0.5,0.6],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [ 5,6,7,8,9,10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1,2,3,4,5,6,7,8,9,10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

joblib_path_suivi_metrique_1 = get_output_path_file("df_score_random_v1.joblib")
joblib_path_suivi_metrique_2 = get_output_path_file("df_score_best_test_v1.joblib")

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Modèle instancié
model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.1, objective="multi:softprob")

randomized_search_CV = RandomizedSearchCV(model, param_distributions=param_grid,n_iter=2000, cv=5, scoring='neg_log_loss',n_jobs=-1)
df_score_random_v1,df_score_best_test_v1 = test_randomized_model(joblib_path_suivi_metrique_1, joblib_path_suivi_metrique_2, param_grid_ameliore, model, randomized_search_CV)

Création de nouveaux DataFrames
Début du randomized: RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   param_distributions={'booster': ['gbtree'],
                      

In [47]:
df_score_random_v1

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std
0,250,9,0.01,0.1,0.75,1,10,1,1,multi:softprob,mlogloss,gbtree,-0.495877,0.007797
1,100,3,0.2,0.5,1,1,1,0.1,10,multi:softprob,mlogloss,gbtree,-0.448388,0.014140
2,1000,9,0.05,1,0.5,0.5,10,1,10,multi:softprob,mlogloss,gbtree,-0.440227,0.015089
3,500,12,0.2,0,0.75,0.75,5,0,0,multi:softprob,mlogloss,gbtree,-0.619045,0.030493
4,500,9,0.2,1,0.5,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.466344,0.015956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18495,250,3,0.2,0.5,0.75,0.75,5,0,0,multi:softprob,mlogloss,gbtree,-0.457141,0.016801
18496,50,9,0.01,1,0.75,0.5,10,1,0,multi:softprob,mlogloss,gbtree,-0.814215,0.002704
18497,500,9,0.2,0.1,0.75,1,10,1,0,multi:softprob,mlogloss,gbtree,-0.521037,0.020914
18498,100,9,0.01,0,1,0.75,10,0,0,multi:softprob,mlogloss,gbtree,-0.644294,0.005311


In [48]:
df_score_best_test_v1

,Best_param,LogLoss,Accuracy,F1_weighted,F1_macro,Duree en sec randomized
0,"{'subsample': 0.5, 'reg_lambda': 0, 'reg_alpha...",0.448550,0.826692,0.815207,0.610022,3441.852242
1,"{'subsample': 0.75, 'reg_lambda': 0, 'reg_alph...",0.453031,0.822264,0.810324,0.606093,3966.272579
2,"{'subsample': 0.75, 'reg_lambda': 1, 'reg_alph...",0.450924,0.826692,0.816315,0.625979,966.120473
3,"{'subsample': 0.5, 'reg_lambda': 0, 'reg_alpha...",0.447097,0.824162,0.812921,0.607456,3853.032981
4,"{'subsample': 0.5, 'reg_lambda': 1, 'reg_alpha...",0.448739,0.824794,0.812801,0.598961,3708.291542
5,"{'subsample': 0.5, 'reg_lambda': 1, 'reg_alpha...",0.447176,0.824794,0.813524,0.607785,18971.756044


In [49]:
from joblib import load,dump
dump(df_score_best_test_v1,"df_score_best_test_v1.joblib")

['df_score_best_test.joblib']

In [50]:
dump(df_score_random_v1,"df_score_random_v1.joblib")

['df_score_random.joblib']

In [23]:
df_score_random = load("df_score_random.joblib")
df_score_random

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std
0,250,9,0.01,0.1,0.75,1,10,1,1,multi:softprob,mlogloss,gbtree,-0.495877,0.007797
1,100,3,0.2,0.5,1,1,1,0.1,10,multi:softprob,mlogloss,gbtree,-0.448388,0.014140
2,1000,9,0.05,1,0.5,0.5,10,1,10,multi:softprob,mlogloss,gbtree,-0.440227,0.015089
3,500,12,0.2,0,0.75,0.75,5,0,0,multi:softprob,mlogloss,gbtree,-0.619045,0.030493
4,500,9,0.2,1,0.5,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.466344,0.015956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,250,6,0.05,0,0.75,1,10,0,0,multi:softprob,mlogloss,gbtree,-0.444509,0.014346
4496,100,3,0.05,0.5,1,1,10,0,10,multi:softprob,mlogloss,gbtree,-0.477466,0.006022
4497,250,3,0.1,1,0.75,1,5,1,1,multi:softprob,mlogloss,gbtree,-0.441659,0.014930
4498,1000,6,0.2,0.1,0.75,0.5,10,0.1,0,multi:softprob,mlogloss,gbtree,-0.565322,0.022599


In [24]:
df_score_best_test=load("df_score_best_test.joblib")
df_score_best_test

,Best_param,LogLoss,Accuracy,F1_weighted,F1_macro,Duree en sec randomized
0,"{'subsample': 0.5, 'reg_lambda': 0, 'reg_alpha...",0.448550,0.826692,0.815207,0.610022,3441.852242
1,"{'subsample': 0.75, 'reg_lambda': 0, 'reg_alph...",0.453031,0.822264,0.810324,0.606093,3966.272579
2,"{'subsample': 0.75, 'reg_lambda': 1, 'reg_alph...",0.450924,0.826692,0.816315,0.625979,966.120473
